In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from dgp import DgpProportion, ExperimentImpact
from design import Sequential

In [3]:
design = Sequential(delta=0.10)
N, d_star = design.fixed_wins_proportion()

N:2922, d*: 106


In [4]:
expt = ExperimentImpact(perc_change=[0,20])
dgp = DgpProportion(prob_success=0.2)
stream_df = dgp.dgp(n_units=N*10, expt_impact=expt) # conservative estimate to reach t+c>=N
stream_df['t_minus_c'] = (((-1)**(stream_df.treatment+1)) * stream_df.outcome).cumsum()
stream_df['t_plus_c'] = stream_df.outcome.cumsum()
stream_df

,unit_id,cluster_id,success_prob,outcome,treatment,t_minus_c,t_plus_c
0,0,0,0.24,0,1,0,0
1,1,0,0.24,0,1,0,0
2,2,0,0.24,0,1,0,0
3,3,0,0.24,1,1,1,1
4,4,0,0.24,0,1,1,1
...,...,...,...,...,...,...,...
29215,29215,0,0.24,0,1,718,6644
29216,29216,0,0.20,1,0,717,6645
29217,29217,0,0.20,0,0,717,6645
29218,29218,0,0.24,0,1,717,6645


In [6]:
effect_found = 0
n_sims = 1_000
for _ in range(n_sims):
	expt = ExperimentImpact(perc_change=[0,0])
	dgp = DgpProportion(prob_success=0.2)
	stream_df = dgp.dgp(n_units=N*10, expt_impact=expt)
	stream_df['t_minus_c'] = (((-1)**(stream_df.treatment+1)) * stream_df.outcome).cumsum()
	stream_df['t_plus_c'] = stream_df.outcome.cumsum()
	effect_found += any((stream_df.t_minus_c==d_star) & (stream_df.t_plus_c<N))
print(effect_found/n_sims)

0.051
